
<img align="right" width="125" src="https://www.ou.nl/documents/40554/3255217/Logo_OU.jpg"/>

<br>


<center> <font size ="6" color='red'> Machine Learning Training Dashboard </font></center>
<br> 

<br>

Model Development dashboard 


In [11]:
#### @title ML Dashboard
from datetime import datetime
import io
from IPython.display import clear_output,HTML, display
from ipytree import Tree, Node
from ipywidgets import *
import warnings
import os

online_version = True
warnings.filterwarnings("ignore")
drive = None

if online_version:
    display(HTML("""
    <style>
    .status-todo .p-Collapse-header {
        background-color: lightyellow;
    }

    .status-done .p-Collapse-header {
        background-color: lightgreen;
    }
                  
    .status-inprogress .p-Collapse-header {
        background-color: lightskyblue;
    }
    </style>
    """))
    
    !git clone https://github.com/muratfirat78/ML_Dashboard &> /dev/null
    os.chdir('/content/ML_Dashboard')

    from google.colab import auth
    from controller.controller import Controller
    from googleapiclient.discovery import build
    import googleapiclient.http
    import random
    import numpy as np
    
    class GoogleDrive:
      def __init__(self):
          auth.authenticate_user()
          self.drive_service = build('drive', 'v3')
          self.folderid = '1pN1jFF5tcDxrfQRdTQrHDNvvxRLRbI69'
          self.userid = None
    
      def get_folder(self, userid):
        query = f"name='{userid}' and '{self.folderid}' in parents and mimeType='application/vnd.google-apps.folder' and trashed=false"
        results = self.drive_service.files().list(q=query, fields="files(id)").execute()
        files = results.get('files', [])
    
        if files:
            return files[0]['id']
      
      def download(self, file_id, file_name, userid):
        request = self.drive_service.files().get_media(fileId=file_id)

        #create the userid folder if not existing
        path = os.path.join('drive', str(userid))
        os.makedirs(path, exist_ok=True)

        file_path = os.path.join(path, file_name)

        fh = io.FileIO(f"{file_path}", 'wb')
        downloader = googleapiclient.http.MediaIoBaseDownload(fh, request)
        done = False
        while done is False:
          status, done = downloader.next_chunk()

      def get_performances(self,userid):
        folderid = self.get_folder(userid)
        query = f"'{folderid}' in parents and trashed=false"
        results = self.drive_service.files().list(q=query, fields="files(id, name)").execute()
        files = results.get('files', [])
        
        if not os.path.exists('/content/ML_Dashboard/drive/' + str(userid)):
            os.makedirs('/content/ML_Dashboard/drive/' + str(userid))
          
        for file in files:
          if not os.path.exists('./drive/' + userid + '/' + file['name']):
            self.download(file['id'], file['name'], userid)
    
      def login_correct(self,userid):
        query = f"name='{userid}' and '{self.folderid}' in parents and mimeType='application/vnd.google-apps.folder' and trashed=false"
        results = self.drive_service.files().list(q=query, fields="files(id)").execute()
        if len(results['files']) > 0:
          return True
        else:
          return False

      def register(self):
        if self.userid !=None:
          return "User already exists, your username is: " + self.userid

        query = f"'{self.folderid}' in parents and mimeType='application/vnd.google-apps.folder' and trashed = false and 'me' in owners"
        results = self.drive_service.files().list(q=query,pageSize=1000,fields='files(id, name)').execute()
        files = results.get('files', [])

        if len(files) > 0:
          return "User exists, your username is: " + files[0]['name']
        else:
          userid = None
          while True:
            userid = str(random.randint(10000, 99999))
            query2 = f"name='{userid}' and '{self.folderid}' in parents and mimeType='application/vnd.google-apps.folder' and trashed=false"
            results2 = self.drive_service.files().list(q=query2,pageSize=1000,fields="files(name)").execute()
            files2 = results2.get('files', [])
            if len(files2) > 0:
              continue
            else:
              file_metadata = {
                'name': userid,
                'mimeType': 'application/vnd.google-apps.folder',
                'parents': [self.folderid]
              }
              file = self.drive_service.files().create(body=file_metadata, fields='id').execute()
              self.userid = userid
              return "User created, your userid is: " + userid
        
    
      def to_serializable(self, obj):
          if isinstance(obj, np.generic):
              return obj.item()
          elif isinstance(obj, dict):
              return {k: self.to_serializable(v) for k, v in obj.items()}
          elif isinstance(obj, list):
              return [self.to_serializable(v) for v in obj]
          elif isinstance(obj, tuple):
              return tuple(self.to_serializable(v) for v in obj)
          else:
              return obj


      def upload_log(self, result, userid, timestamp):
        with open('./drive/'+ userid + '/' + timestamp +
                  '.txt', 'w') as f:
          f.write(str(self.to_serializable(result)))
    
        folderid = self.get_folder(userid)

        #see if the file already exists
        query = f"name='{timestamp}.txt' and '{folderid}' in parents and trashed = false"
        response = self.drive_service.files().list(q=query, spaces='drive', fields='files(id)').execute()
        files = response.get('files', [])
    
        media = googleapiclient.http.MediaFileUpload('./drive/'+ userid + '/' + timestamp + '.txt', mimetype="text/plain", resumable=True)

        if files:
            #file already exists, overwrite
            file_id = files[0]['id']
            uploaded_file = self.drive_service.files().update(
                fileId=file_id,
                media_body=media
            ).execute()
        else:
            #file does not exist yet, create
            file_metadata = {
            "name": timestamp + ".txt",
            "parents": [folderid]
            }
            
            uploaded_file = self.drive_service.files().create(
                body=file_metadata,
                media_body=media,
                fields="id"
            ).execute()
    
    drive = GoogleDrive()
    
    from controller.controller import Controller
    controller = Controller(drive, online_version)
else:
    from controller.controller import Controller
    controller = Controller(drive, online_version)

ui = controller.get_ui()

ui